In [12]:
import pandas as pd
df = pd.read_csv('./winequality-white.dat')
df.head()

,FixedAcidity,VolatileAcidity,CitricAcid,ResidualSugar,Chlorides,FreeSulfurDioxide,TotalSulfurDioxide,Density,PH,Sulphates,Alcohol,Quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [13]:
from sklearn.utils import shuffle

df = shuffle(df, random_state=42).iloc[:2000, :].reset_index(drop=True)
df.head()

,FixedAcidity,VolatileAcidity,CitricAcid,ResidualSugar,Chlorides,FreeSulfurDioxide,TotalSulfurDioxide,Density,PH,Sulphates,Alcohol,Quality
0,6.0,0.29,0.41,10.8,0.048,55.0,149.0,0.99370,3.09,0.59,10.966667,7
1,5.4,0.53,0.16,2.7,0.036,34.0,128.0,0.98856,3.20,0.53,13.200000,8
2,7.1,0.25,0.39,2.1,0.036,30.0,124.0,0.99080,3.28,0.43,12.200000,8
3,7.3,0.28,0.35,1.6,0.054,31.0,148.0,0.99178,3.18,0.47,10.700000,5
4,6.5,0.32,0.34,5.7,0.044,27.0,91.0,0.99184,3.28,0.60,12.000000,7


In [14]:
# Все типы данных подобраны правильно
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   FixedAcidity         2000 non-null   float64
 1    VolatileAcidity     2000 non-null   float64
 2    CitricAcid          2000 non-null   float64
 3    ResidualSugar       2000 non-null   float64
 4    Chlorides           2000 non-null   float64
 5    FreeSulfurDioxide   2000 non-null   float64
 6    TotalSulfurDioxide  2000 non-null   float64
 7    Density             2000 non-null   float64
 8    PH                  2000 non-null   float64
 9    Sulphates           2000 non-null   float64
 10   Alcohol             2000 non-null   float64
 11   Quality             2000 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 187.6 KB


In [15]:
#Стандартизация данных
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(df.iloc[:, :-1])
y = df[' Quality']
x

array([[-1.01156131,  0.14813905,  0.60241992, ..., -0.63382776,
         0.87848637,  0.37230182],
       [-1.72932005,  2.57706291, -1.45404188, ...,  0.09330426,
         0.3535468 ,  2.19277553],
       [ 0.3043297 , -0.25668159,  0.43790298, ...,  0.62212755,
        -0.52135248,  1.37763805],
       ...,
       [-0.7723084 ,  0.85657518,  0.93145381, ...,  0.82043628,
         0.61601658, -0.57869191],
       [-0.05454966,  4.34815323, -0.71371563, ...,  1.15095084,
         0.26605687, -0.82323316],
       [ 1.62022072,  0.14813905,  1.2604877 , ...,  0.3577159 ,
        -1.13378198,  0.31795932]])

In [16]:
#Делим выборку на обучающую и валидационную и тестовую
from sklearn.model_selection import train_test_split
x_train, x_temp, y_train, y_temp = train_test_split(x, y, 
                                                    test_size=0.5, 
                                                    random_state=2)

x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, 
                                                    test_size=0.4, 
                                                    random_state=2)
x_train.shape[0], x_val.shape[0], x_test.shape[0]

(1000, 600, 400)

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score

ccp_alpha  = [0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.2, 0.8]
criterion = ['entropy', 'gini']

df_temp = []
for crit in criterion:
    for alpha in ccp_alpha:
        temp = []
        temp.append(alpha)
        temp.append(crit)
        tree = DecisionTreeClassifier(ccp_alpha=alpha, criterion=crit)
        tree.fit(x_train, y_train)
        y_pred = tree.predict(x_test)
        temp.append(accuracy_score(y_test, y_pred))
        temp.append(recall_score(y_test, y_pred, average='micro'))
        temp.append(precision_score(y_test, y_pred, average='micro'))
        temp.append(roc_auc_score(y_test, tree.predict_proba(x_test), average='macro', multi_class="ovr"))
        df_temp.append(temp)

tree_scores = pd.DataFrame(df_temp)
tree_scores.columns = ['alpha', 'criterion', "accuracy_score", 'recall_score', 'precision_score', 'roc_auc_score']
tree_scores.sort_values(by=['accuracy_score','recall_score',	'precision_score',	'roc_auc_score'], ascending=False)
tree_scores
# Best params: Criterion : entropy, Alpha : 0.005

,alpha,criterion,accuracy_score,recall_score,precision_score,roc_auc_score
0,0.005,entropy,0.4775,0.4775,0.4775,0.590909
1,0.010,entropy,0.4625,0.4625,0.4625,0.628403
2,0.015,entropy,0.4900,0.4900,0.4900,0.642893
3,0.020,entropy,0.4575,0.4575,0.4575,0.634479
4,0.025,entropy,0.4575,0.4575,0.4575,0.632167
5,0.030,entropy,0.4575,0.4575,0.4575,0.632167
6,0.035,entropy,0.5050,0.5050,0.5050,0.598570
7,0.200,entropy,0.4475,0.4475,0.4475,0.500000
8,0.800,entropy,0.4475,0.4475,0.4475,0.500000
9,0.005,gini,0.4875,0.4875,0.4875,0.591997


In [8]:
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn.svm import SVC
from tqdm import tqdm

krls  = ['linear', 'rbf', 'sigmoid'] # poly
gammas = ['scale', 'auto', 1, 10, 0.1, 20, 0.01]
coef0s = [0, 1, 2, 5, 10, 0.1, 0.01]
degrees = [2, 3, 4, 5]
cs = [0.001, 0.01, 0.1, 0.5, 1, 2, 10, 20]

df_temp = []
for krl in tqdm(krls):
    print(f"krl = {krl}")
    for gamma in gammas:
        print(f"gamma = {gamma}")
        for coef0 in coef0s:
            for degree in degrees:
                for c in cs:
                    temp = []
                    temp.append(krl)
                    temp.append(gamma)
                    temp.append(coef0)
                    temp.append(degree)
                    temp.append(c)
                    clf = SVC(kernel=krl, gamma=gamma, coef0=coef0, degree=degree, C=c, probability=True, cache_size=6000)
                    clf.fit(x_train, y_train)
                    y_pred = clf.predict(x_test)
                    temp.append(accuracy_score(y_test, y_pred))
                    temp.append(recall_score(y_test, y_pred, average='micro'))
                    temp.append(precision_score(y_test, y_pred, average='micro'))
                    temp.append(roc_auc_score(y_test, clf.predict_proba(x_test), average='macro', multi_class="ovr"))
                    df_temp.append(temp)

svc_scores = pd.DataFrame(df_temp)
svc_scores.columns = ['kernel', 'gamma', "coef0", "degree", "C" ,"accuracy_score", 'recall_score', 'precision_score', 'roc_auc_score']
# svc_scores.sort_values(by=['accuracy_score','recall_score',	'precision_score',	'roc_auc_score'], ascending=False)
svc_scores
# Best params: kernel = rbf, gamma = auto, coef0 = 0, degree = 2, C = 0.5

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
  0%|          | 0/3 [00:00<?, ?it/s]

krl = linear
gamma = scale
gamma = auto
gamma = 1
gamma = 10
gamma = 0.1
gamma = 20
gamma = 0.01


 33%|███▎      | 1/3 [05:20<10:40, 320.21s/it]

krl = rbf
gamma = scale
gamma = auto
gamma = 1
gamma = 10
gamma = 0.1
gamma = 20
gamma = 0.01


 67%|██████▋   | 2/3 [07:51<03:40, 220.99s/it]

krl = sigmoid
gamma = scale
gamma = auto
gamma = 1
gamma = 10
gamma = 0.1
gamma = 20
gamma = 0.01


100%|██████████| 3/3 [16:51<00:00, 337.32s/it]


,kernel,gamma,coef0,degree,C,accuracy_score,recall_score,precision_score,roc_auc_score
0,linear,scale,0.00,2,0.001,0.4475,0.4475,0.4475,0.679755
1,linear,scale,0.00,2,0.010,0.5100,0.5100,0.5100,0.679349
2,linear,scale,0.00,2,0.100,0.5400,0.5400,0.5400,0.638677
3,linear,scale,0.00,2,0.500,0.5450,0.5450,0.5450,0.692893
4,linear,scale,0.00,2,1.000,0.5450,0.5450,0.5450,0.689747
...,...,...,...,...,...,...,...,...,...
4699,sigmoid,0.01,0.01,5,0.500,0.4900,0.4900,0.4900,0.725199
4700,sigmoid,0.01,0.01,5,1.000,0.5100,0.5100,0.5100,0.722480
4701,sigmoid,0.01,0.01,5,2.000,0.5325,0.5325,0.5325,0.720763
4702,sigmoid,0.01,0.01,5,10.000,0.5475,0.5475,0.5475,0.713719


In [11]:
svc_scores.sort_values(by=['accuracy_score','recall_score',	'precision_score',	'roc_auc_score'], ascending=False)

,kernel,gamma,coef0,degree,C,accuracy_score,recall_score,precision_score,roc_auc_score
1795,rbf,auto,0.0,2,0.5,0.5600,0.5600,0.5600,0.723390
1803,rbf,auto,0.0,3,0.5,0.5600,0.5600,0.5600,0.723390
1811,rbf,auto,0.0,4,0.5,0.5600,0.5600,0.5600,0.723390
1819,rbf,auto,0.0,5,0.5,0.5600,0.5600,0.5600,0.723390
1827,rbf,auto,1.0,2,0.5,0.5600,0.5600,0.5600,0.723390
...,...,...,...,...,...,...,...,...,...
3693,sigmoid,1,5.0,3,2.0,0.3125,0.3125,0.3125,0.497894
3683,sigmoid,1,5.0,2,0.5,0.3100,0.3100,0.3100,0.546332
3707,sigmoid,1,5.0,5,0.5,0.3100,0.3100,0.3100,0.525934
3699,sigmoid,1,5.0,4,0.5,0.3100,0.3100,0.3100,0.516708


In [18]:
tree = DecisionTreeClassifier(ccp_alpha=0.005, criterion='entropy')
tree.fit(x_train, y_train)
y_pred = tree.predict(x_val)
print(accuracy_score(y_val, y_pred))
print(recall_score(y_val, y_pred, average='micro'))
print(precision_score(y_val, y_pred, average='micro'))
print(roc_auc_score(y_val, tree.predict_proba(x_val), average='macro', multi_class="ovr"))

0.505
0.505
0.505
0.606167855431282


In [19]:
clf = SVC(kernel='rbf', gamma='auto', coef0=0, degree=2, C=0.5, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_val)
print(accuracy_score(y_val, y_pred))
print(recall_score(y_val, y_pred, average='micro'))
print(precision_score(y_val, y_pred, average='micro'))
print(roc_auc_score(y_val, clf.predict_proba(x_val), average='macro', multi_class="ovr"))

0.5383333333333333
0.5383333333333333
0.5383333333333333
0.7635380689911028
